In [1]:
!pip install flashtext
!pip install scikit-learn
!pip install matplotlib

You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import re
import pandas as pd
from helpers import yake_helper
from datetime import datetime, timedelta
from math import sqrt, floor
from sklearn.cluster import SpectralClustering
import numpy as np
import itertools
from matplotlib import pyplot as plt
from remove_polite_posts import remove_polite_posts as rpp
from flashtext.keyword import KeywordProcessor
import string
import nltk
import math
import warnings

warnings.filterwarnings('ignore')

In [3]:
forum_posts = pd.read_csv('ForumMessages.csv')

w2v_vectors = pd.read_csv('fine-tuning-word2vec/forum_posts_word2vec.model',
                          delim_whitespace=True,
                          skiprows=[0],
                          header=None)

In [4]:
forum_posts.head()

,Id,ForumTopicId,PostUserId,PostDate,ReplyToForumMessageId,Message,Medal,MedalAwardDate
0,28,16,368,05/12/2010 04:39:30,NaN,Here are some papers that analyze Eurovision v...,NaN,NaN
1,29,16,368,05/12/2010 04:39:30,NaN,More research... enjoy<br><br>Love thy Neighbo...,NaN,NaN
2,993,133,2691,10/10/2010 02:39:04,NaN,<div>@Nan</div><div><br></div><div>Isn't X_i s...,NaN,NaN
3,996,133,2958,10/10/2010 02:39:04,NaN,"Brad,<br>It is not. Y_i is defined as I (S_{i+...",NaN,NaN
4,1350,214,4284,12/01/2010 05:54:28,NaN,After I specify the adress of my result file a...,NaN,NaN


In [5]:
w2v_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,p,-0.258226,1.322337,0.291533,-0.679372,0.362768,0.961267,0.694033,-0.019635,-0.007253,...,0.263582,-0.606414,0.356946,-0.168584,-0.717055,-1.616924,-0.834215,-0.698412,-0.177316,-0.203391
1,the,1.273628,-0.651237,-0.065435,-0.083179,1.152159,-0.257050,1.377131,1.871087,0.498178,...,0.166710,-1.062766,1.464719,-0.346945,-0.520129,-1.788661,-0.298106,-0.374428,0.806349,-2.619995
2,a,0.643546,-1.170918,0.753471,0.290148,0.276544,1.110373,3.050881,1.799117,-0.873134,...,0.684108,-0.937324,0.919357,0.979155,-1.046691,-1.881935,0.662633,0.407499,1.435630,-1.053839
3,i,1.832707,-0.055822,0.288658,0.215443,2.397752,2.606470,-1.467209,1.140652,-0.241518,...,-0.217198,1.053774,1.247309,0.098720,0.297497,-2.680148,-1.205078,0.714816,0.450466,-1.331006
4,to,2.431875,0.986299,-1.254033,0.172852,0.473982,-0.299878,1.807054,0.864406,-1.208238,...,0.171002,0.671840,0.925534,-1.304466,0.659252,-0.420799,0.295289,0.453520,0.407416,-2.202719


In [6]:
w2v_vectors.index = w2v_vectors[0]
w2v_vectors.drop(0, axis=1, inplace=True)

In [7]:
def vectors_from_post(post):
    all_words = []
    for word in post:
        all_words.append(word)
    return(w2v_vectors[w2v_vectors.index.isin(all_words)])

def doc_embed_from_post(post):
    test_vectors = vectors_from_post(post)
    return(test_vectors.mean())

def get_keyword_set_by_cluster(number):
    cluster_index = list(clustering.labels_ == number)
    return(list(itertools.compress(keyword_sets, cluster_index)))

def get_post_info_by_cluster(number, data, cluster):
    return(data[cluster.labels_ == number])

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return(re.sub(clean, '', text))

def remove_thanks(text):
    text = text.lower()
    text = re.sub("nice", "", text)
    text = re.sub("thank.*\s", " ", text)
    text = re.sub("good","", text)
    text = re.sub("hi", "", text)
    text = re.sub("hello", "", text)
    return (text)

def polite_post_index(forum_posts):
    polite_indexes = []
    
    stop_word_list = ["no problem", "thanks", "thx", "thank", "great",
                      "nice", "interesting", "awesome", "perfect", 
                      "amazing", "well done", "good job"]

    keyword_processor = KeywordProcessor()
    keyword_processor.add_keywords_from_list(stop_word_list)

    for i,post in enumerate(forum_posts):
        post = post.lower().translate(str.maketrans({a:None for a in string.punctuation}))        
        if len(post) < 100:
            keywords_found = keyword_processor.extract_keywords(post.lower(), span_info=True)
            if keywords_found:
                polite_indexes.append(i)

    return (polite_indexes)

In [8]:
# Hyperparameter
days_of_posts = 21

In [9]:
forum_topics = pd.read_csv('ForumTopics.csv').rename(columns={'Title': 'TopicTitle'})
forums = pd.read_csv('Forums.csv').rename(columns={'Title': 'ForumTitle'})

In [10]:
forum_topics.head()

,Id,ForumId,KernelId,LastForumMessageId,FirstForumMessageId,CreationDate,LastCommentDate,TopicTitle,IsSticky,TotalViews,Score,TotalMessages,TotalReplies
0,9777,15,NaN,50659.0,50659.0,07/16/2014 05:36:18,07/16/2014 05:36:18,Anyone interested in deep learning?,False,719,0,1,0
1,9787,15,NaN,50731.0,50720.0,07/16/2014 18:19:13,07/16/2014 19:55:52,feature request,False,789,1,2,1
2,9791,15,NaN,50848.0,50749.0,07/17/2014 04:38:08,07/18/2014 14:03:45,continuous dependent variable and few continuo...,False,3964,1,5,4
3,9806,15,NaN,50841.0,50841.0,07/18/2014 13:07:13,07/18/2014 13:07:13,need to find a dataset on news articels shared...,False,566,0,1,0
4,9814,15,NaN,98987.0,50896.0,07/19/2014 14:40:50,11/17/2015 19:14:39,Data Scientists in New York City,False,1200,0,4,3


In [11]:
forums.head()

,Id,ParentForumId,ForumTitle
0,28121,1023.0,pokemon1
1,28128,1023.0,Poverty Rate of Indian States
2,28146,1023.0,Sales Of Shampoo
3,28164,1023.0,submission
4,28178,1023.0,mediumeda


In [12]:
df1 = pd.merge(forum_posts[['ForumTopicId', 'PostDate', 'Message']], forum_topics[['Id', 'ForumId', 'TopicTitle']], left_on='ForumTopicId', right_on='Id')
df1 = df1.drop(['ForumTopicId', 'Id'], axis=1)

forum_posts = pd.merge(df1, forums[['Id', 'ForumTitle']], left_on='ForumId', right_on='Id')
forum_posts = forum_posts.drop(['ForumId', 'Id'], axis=1)
forum_posts.head()

,PostDate,Message,TopicTitle,ForumTitle
0,05/12/2010 04:39:30,Here are some papers that analyze Eurovision v...,Studies that look at Eurovision voting patterns,Forecast Eurovision Voting
1,05/12/2010 04:39:30,More research... enjoy<br><br>Love thy Neighbo...,Studies that look at Eurovision voting patterns,Forecast Eurovision Voting
2,05/12/2010 08:34:27,There's also this piece from the BBC on the <a...,The maths of Eurovision voting,Forecast Eurovision Voting
3,10/10/2010 02:39:04,<div>@Nan</div><div><br></div><div>Isn't X_i s...,And the winner is ...,INFORMS Data Mining Contest 2010
4,10/10/2010 02:39:04,"Brad,<br>It is not. Y_i is defined as I (S_{i+...",And the winner is ...,INFORMS Data Mining Contest 2010


In [13]:
forum_posts.tail(10)

,PostDate,Message,TopicTitle,ForumTitle
1330421,10/26/2021 21:42:01,"<p>Two recent studies, published in Science, s...",Reciprocate or Retaliate. Strategic Decisions ...,Skills Survey
1330422,10/31/2021 14:22:57,<p>Vi no site do CBIC que tem mais de 15datase...,Outros datasets relacionados,CUB Médio Brasil
1330423,11/01/2021 20:24:58,"<p><a href=""https://www.kaggle.com/jeanpauldos...",Outros datasets relacionados,CUB Médio Brasil
1330424,10/28/2021 17:58:49,<p>These data are collected as a part of the G...,About the data,globalPetSurvey
1330425,10/30/2021 19:12:48,"<p>Thank you for the data, Shib is crazy these...",Thx for the update,SHIBA INU Data
1330426,10/31/2021 06:35:12,<p>The detail mentioned for attribute column '...,Clarification of attribute- Target,Tour & Travels Customer Churn Prediction
1330427,10/31/2021 09:51:25,"<p>Oh sorry, made a mistake.<br>\n1 is For Cus...",Clarification of attribute- Target,Tour & Travels Customer Churn Prediction
1330428,10/31/2021 09:59:26,<p>Thank you.</p>,Clarification of attribute- Target,Tour & Travels Customer Churn Prediction
1330429,11/01/2021 07:41:05,<p>I was working on the dataset provided but I...,Column descriptions,Stack Overflow Annual Developer Survey 2021
1330430,10/31/2021 10:38:21,<p>what are probe.csv and df.csv files?</p>,probe.csv and df.cvs,Liver and Liver Tumor Segmentation


In [14]:
# Sample posts with days_of_posts as a hyperparameter

forum_posts['Date'] = pd.to_datetime(forum_posts.PostDate, format='%m/%d/%Y %H:%M:%S')
start_time = datetime.now() + timedelta(days=-days_of_posts) 

sample_post_info = forum_posts.loc[forum_posts.Date > start_time]
sample_posts = sample_post_info.Message.astype('str')

sample_post_info.reset_index(drop=True)
sample_posts.reset_index(drop=True)

0            <p>Thanks for sharing, this is helpful :)</p>
1        <p>A way to filter ranking and non-ranking com...
2        <p><a href="https://www.kaggle.com/ashminer" t...
3                                   <p>got it. Thanks!</p>
4        <p>I have a Russian number, too. For those who...
                               ...                        
14244    <p>The detail mentioned for attribute column '...
14245    <p>Oh sorry, made a mistake.<br>\n1 is For Cus...
14246                                    <p>Thank you.</p>
14247    <p>I was working on the dataset provided but I...
14248          <p>what are probe.csv and df.csv files?</p>
Name: Message, Length: 14249, dtype: object

In [15]:
sample_posts.shape

(14249,)

In [16]:
total_posts = sample_posts.shape[0]
total_clusters = floor(sqrt(total_posts))
print(total_clusters)

119


In [17]:
keywords_tokenized = yake_helper.tokenizing_after_YAKE(sample_posts)
keyword_sets = [set(post) for post in keywords_tokenized]

In [18]:
keyword_sets[0]

{'for', 'helpful', 'is', 'p', 'sharing', 'thanks', 'this'}

In [19]:
document_embeddings = np.zeros([total_posts, 300])

for i in range(total_posts):
    embeddings = np.array(doc_embed_from_post(keyword_sets[i]))
    if np.isnan(embeddings).any():
        document_embeddings[i,:] = np.zeros([1,300])
    else:
        document_embeddings[i,:] = embeddings

In [ ]:
clustering = SpectralClustering(n_clusters=total_clusters,
                                assign_labels="discretize",
                                n_neighbors=total_clusters).fit(document_embeddings)

In [ ]:
clustering.labels_

In [ ]:
pd.Series(clustering.labels_).value_counts()

In [ ]:
for i in range(total_clusters):
    print(f"Cluster {i} \n")
    print(get_post_info_by_cluster(i, sample_post_info, clustering))
    print("\n")

In [ ]:
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
posts_as_string = sample_post_info.Message.to_string(index=False)

posts_as_string = remove_thanks(posts_as_string)

wordcloud = WordCloud().generate(posts_as_string)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()